<h2>Imports</h2>

In [41]:
# Import libraries

import simpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from util import Battery, read_battery_process, solar_charge_process, compute_tasks_process, wind_charge_process

<h2>Constraints</h2>

In [48]:
# Set constraints

minimal_reliability = 0.95
season = 'summer'

<h2>Data prep</h2>

In [43]:
# Load and prepare data from csv-files
START_DATE = pd.to_datetime("2021-06-01 00:00:00")
END_DATE = pd.to_datetime("2021-06-08 00:00:00")
INDEX = pd.date_range(START_DATE, END_DATE, freq="s")  # +timedelta(minutes=1)


# Load data between START_DATE and END_DATE
with open("data/weather_berlin_2021-06.csv", "r") as csvfile:
    weather_df = pd.read_csv(csvfile, parse_dates=True, index_col=0)[START_DATE:END_DATE]
with open("data/low_prio_tasks.csv", "r") as csvfile:
    tasks_df = pd.read_csv(csvfile, parse_dates=True, index_col=0)[START_DATE:END_DATE]
with open("data/weather_berlin_2021-12.csv", "r") as csvfile:
    weather_winter_df = pd.read_csv(csvfile, parse_dates=True, index_col=0)[START_DATE:END_DATE]


<h2>Scenario and costs</h2>

In [1]:
# Set researched values for costs etc. depending on the scenario

# BATTERIES
BATTERY_CAPACITIES = [i for i in range(500, 10500, 500)]    # Wh
cost_per_capacity = 1                                       # €/Wh

# SOLAR PANELS
SOLAR_AREAS = [i*1.7 for i in range(4)]                     # m^2
cost_per_solar = 330                                        # €/Panel

# WIND TURBINE
wind_areas = ["large", "small", "none"]
wind_prices = [400, 2000]                                   # €



"""
Sources:

https://www.photovoltaik-shop.com/solarmodul-meyer-burger-black-380wp-mono-halbzellen-full-black-2006000018.html
https://de.exchange-rates.org/Rate/USD/EUR/16.12.2020
https://www.ebay.de/itm/234775191377?_trkparms=amclksrc%3DITM%26aid%3D1110006%26algo%3DHOMESPLICE.SIM%26ao%3D1%26asc%3D243355%26meid%3D0b17340d58b649be96fbdc4667439b63%26pid%3D101224%26rk%3D2%26rkt%3D5%26sd%3D234632728658%26itm%3D234775191377%26pmt%3D0%26noa%3D1%26pg%3D2047675%26algv%3DDefaultOrganicWebWithV5RefreshRanker&_trksid=p2047675.c101224.m-1
https://www.omnicalculator.com/ecology/wind-turbine#how-to-calculate-the-power-generated-by-a-wind-turbine
https://www.amazon.de/ZHIHUASMTBX-Vertikaler-Windkraftanlage-Verwendung-Geräuschlos/dp/B09SZ4JK22/ref=asc_df_B09SZ4JK22/?tag=googshopde-21&linkCode=df0&hvadid=604161997903&hvpos=&hvnetw=g&hvrand=5782366940088564558&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=9060641&hvtargid=pla-1685653218226&psc=1&th=1&psc=1

"""

'\nSources:\n\nhttps://www.photovoltaik-shop.com/solarmodul-meyer-burger-black-380wp-mono-halbzellen-full-black-2006000018.html\nhttps://de.exchange-rates.org/Rate/USD/EUR/16.12.2020\nhttps://www.ebay.de/itm/234775191377?_trkparms=amclksrc%3DITM%26aid%3D1110006%26algo%3DHOMESPLICE.SIM%26ao%3D1%26asc%3D243355%26meid%3D0b17340d58b649be96fbdc4667439b63%26pid%3D101224%26rk%3D2%26rkt%3D5%26sd%3D234632728658%26itm%3D234775191377%26pmt%3D0%26noa%3D1%26pg%3D2047675%26algv%3DDefaultOrganicWebWithV5RefreshRanker&_trksid=p2047675.c101224.m-1\nhttps://www.omnicalculator.com/ecology/wind-turbine#how-to-calculate-the-power-generated-by-a-wind-turbine\nhttps://www.amazon.de/ZHIHUASMTBX-Vertikaler-Windkraftanlage-Verwendung-Geräuschlos/dp/B09SZ4JK22/ref=asc_df_B09SZ4JK22/?tag=googshopde-21&linkCode=df0&hvadid=604161997903&hvpos=&hvnetw=g&hvrand=5782366940088564558&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=9060641&hvtargid=pla-1685653218226&psc=1&th=1&psc=1\n\n'

<h2>Simulation Function</h2>

In [45]:
def simulation(panel, capacity, area, season):
    """
    Runs simulation for a specific setup.
    """

    # Calculate set-up cost
    cost = cost_per_capacity*capacity + cost_per_solar*panel/1.7
    if area=="large":
        cost += wind_prices[1]
    elif area=="small":
        cost +=wind_prices[0]

    # Initialize Battery
    battery = Battery(capacity=capacity, soc=capacity)
    battery_socs = []
    task_stats = []

    if season=="winter":
        # Run simulation for winter
        env = simpy.Environment()
        env.process(solar_charge_process(env, battery, solar_area=panel, solar_values=weather_winter_df["solar"]))
        env.process(wind_charge_process(env, battery, area, weather_winter_df["wind"]))
        env.process(compute_tasks_process(env, battery, start_date=START_DATE, task_arrival_times=tasks_df.index, task_stats=task_stats))
        env.process(read_battery_process(env, battery, battery_socs))
        env.run(until=len(INDEX))
    else:

        # Run simulation for summer
        env = simpy.Environment()
        env.process(solar_charge_process(env, battery, solar_area=panel, solar_values=weather_df["solar"]))
        env.process(wind_charge_process(env, battery, area, weather_df["wind"]))
        env.process(compute_tasks_process(env, battery, start_date=START_DATE, task_arrival_times=tasks_df.index, task_stats=task_stats))
        env.process(read_battery_process(env, battery, battery_socs))
        env.run(until=len(INDEX))

    # Calculate stats
    rate = check_stats(task_stats)

    return cost, rate



def check_stats(stats):
    """
    Calculates successrate from stats.
    """

    success_count = 0
    for task in stats:
        if task[1]:
            success_count += 1

    return success_count/len(stats)


<h2>Main</h2>

In [50]:
# Initialize resulting DataFrame
columns = ['Panel count', 'Battery capacity', 'Windturbine size', 'Price', 'Success rate']
result_df = pd.DataFrame(columns=columns)

# Running Simulation for all configurations
print(f"STARTING WITH FIRST WIND CONFIG.......")
for count_w, area in enumerate(wind_areas):
    
    for count_p, panel in enumerate(SOLAR_AREAS):

        if count_p ==0:
            print(f"\tSTARTING WITH FIRST SOLAR CONFIG.......")
        for count_b, capacity in enumerate(BATTERY_CAPACITIES):

            cost, rate = simulation(panel, capacity, area, season)

            df_entry = pd.DataFrame([[int(panel/1.7), capacity, area, cost, rate]], columns=columns)
            result_df = pd.concat([result_df, df_entry])
        if count_p != len(SOLAR_AREAS)-1:
            print(f"\tPROGRESSING TO NEXT SOLAR CONFIG ({count_p+2}/{len(SOLAR_AREAS)}).......")

    if count_w != len(wind_areas)-1:
        print(f"PROGRESSING TO NEXT WIND CONFIG ({count_w+2}/{len(wind_areas)}).......")

print("\n____SIMULATION FINISHED_______")

STARTING WITH FIRST WIND CONFIG.......
	STARTING WITH FIRST SOLAR CONFIG.......
	PROGRESSING TO NEXT SOLAR CONFIG (2/4).......
	PROGRESSING TO NEXT SOLAR CONFIG (3/4).......
	PROGRESSING TO NEXT SOLAR CONFIG (4/4).......
PROGRESSING TO NEXT WIND CONFIG (2/3).......
	STARTING WITH FIRST SOLAR CONFIG.......
	PROGRESSING TO NEXT SOLAR CONFIG (2/4).......
	PROGRESSING TO NEXT SOLAR CONFIG (3/4).......
	PROGRESSING TO NEXT SOLAR CONFIG (4/4).......
PROGRESSING TO NEXT WIND CONFIG (3/3).......
	STARTING WITH FIRST SOLAR CONFIG.......
	PROGRESSING TO NEXT SOLAR CONFIG (2/4).......
	PROGRESSING TO NEXT SOLAR CONFIG (3/4).......
	PROGRESSING TO NEXT SOLAR CONFIG (4/4).......

____SIMULATION FINISHED_______


<h2>Results</h2>

In [51]:
# Save results to csv
#path = "/result_summer_highbatteryprice.csv"
#result_df.to_csv(path)


# Calculate best results
over_th = result_df["Success rate"] >= minimal_reliability
results_ok = result_df[over_th]
index = results_ok["Price"].min()
result_cheapest = list(results_ok[results_ok["Price"] == index].to_numpy()[0])

max_succes_rate = result_df["Success rate"].max()
results_ok = result_df[result_df["Success rate"] == max_succes_rate]
index = results_ok["Price"].min()
result_best = list(results_ok[results_ok["Price"] == index].to_numpy()[0])



# Print best system configurations
print("________________________________________________________\n")
print("RESULTS")
print("________________________________________________________\n")
print("The cheapest system to beat the minimal threshold is:\n")
print(f'Number of Solar Panels: {result_cheapest[0]}\n \
    Battery Capacity: {result_cheapest[1]/1000} kWh\n \
    Windturbine Configuration: {result_cheapest[2]}\n \
    Success Rate: {result_cheapest[4]:.3}\n \
    Price: {result_cheapest[3]} €\n\n')

print("The most reliable system to buy is:\n")
print(f'Number of Solar Panels: {result_best[0]}\n \
    Battery Capacity: {result_best[1]/1000} kWh\n \
    Windturbine Configuration: {result_best[2]}\n \
    Success Rate: {result_best[4]:.3}\n \
    Price: {result_best[3]} €\n')
print("________________________________________________________\n")

________________________________________________________

RESULTS
________________________________________________________

The cheapest system to beat the minimal threshold is:

Number of Solar Panels: 3
     Battery Capacity: 0.5 kWh
     Windturbine Configuration: none
     Success Rate: 0.95
     Price: 1490.0 €


The most reliable system to buy is:

Number of Solar Panels: 3
     Battery Capacity: 1.0 kWh
     Windturbine Configuration: none
     Success Rate: 1.0
     Price: 1990.0 €

________________________________________________________

